In [1]:
import os

# Set current working directory to parent folder
os.chdir(os.path.abspath(".."))

## Diarization (No se necesita, ya esta etiquetado)

In [2]:
import json

In [14]:
HUGGINGFACE_ACCESS_TOKEN = json.load(open('config.json'))['HUGGINGFACE_ACCESS_TOKEN']

In [1]:
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token= HUGGINGFACE_ACCESS_TOKEN)

C:\Users\emanu\Anaconda3\envs\SER\lib\site-packages\pyannote\audio\core\io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
C:\Users\emanu\Anaconda3\envs\SER\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\emanu\Anaconda3\envs\SER\lib\site-packages\torch_audiomentations\utils\io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
torchvision is not available - cannot save figures
The torchaudio backend 

In [15]:
# run the pipeline on an audio file
diarization = pipeline("data/MSPCORPUS/Audio/MSP-Conversation_0002.wav")

In [16]:
# dump the diarization output to disk using RTTM format
with open("data/DIARIZATION/MSP-Conversation_0002.rttm", "w") as rttm:
    diarization.write_rttm(rttm)

In [18]:
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

start=0.0s stop=3.6s speaker_SPEAKER_01
start=3.9s stop=6.0s speaker_SPEAKER_01
start=6.2s stop=7.5s speaker_SPEAKER_01
start=7.8s stop=10.6s speaker_SPEAKER_01
start=11.0s stop=14.0s speaker_SPEAKER_01
start=14.4s stop=17.5s speaker_SPEAKER_01
start=17.9s stop=22.0s speaker_SPEAKER_01
start=22.2s stop=23.6s speaker_SPEAKER_01
start=23.7s stop=25.7s speaker_SPEAKER_01
start=26.0s stop=28.0s speaker_SPEAKER_01
start=28.0s stop=30.1s speaker_SPEAKER_00
start=30.4s stop=32.8s speaker_SPEAKER_00
start=33.6s stop=40.4s speaker_SPEAKER_00
start=41.0s stop=43.3s speaker_SPEAKER_00
start=43.7s stop=46.0s speaker_SPEAKER_00
start=46.3s stop=49.2s speaker_SPEAKER_00
start=49.7s stop=55.1s speaker_SPEAKER_00
start=55.4s stop=57.7s speaker_SPEAKER_00
start=58.3s stop=59.7s speaker_SPEAKER_00
start=60.2s stop=60.5s speaker_SPEAKER_01
start=60.5s stop=61.0s speaker_SPEAKER_00
start=61.0s stop=74.4s speaker_SPEAKER_01
start=74.7s stop=82.2s speaker_SPEAKER_01
start=82.6s stop=88.2s speaker_SPEAKER_01

## Transcripción

### Whisper

Transcripciones usando whisper para procesamiento del audio

In [21]:
import whisper

model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 32.6MiB/s]


In [22]:
result = model.transcribe("data/MSPCORPUS/Audio/MSP-Conversation_0002.wav")

In [24]:
result

{'text': " It's our honor to talk about important issues and why they're relevant to each of us. The foundation of our show, Heart of the Matter, focuses on three things. Important issues and why they should be on our radar today. Looking at things as right versus wrong, instead of right versus left. And sharing stories with real people's experiences so that we can brainstorm solutions to problems that we're facing today. We know we can do this together. Thanks for listening and joining the conversation. Kim, how does somebody connect with us today? Well, on Facebook, we are the Americhicks. Our email is Americhicks at Gmail. Our website is americhicks.com. And you can listen live there and call us at 303-477-5600. Molly, we have a really great show plan for you today. Race relations and affirmative action will have Droy Murdoch with us at the half hour. And we saw him at Leadership Program in the Rockies. And it was just a really interesting presentation. And then we will have states 

In [23]:
print(result["text"])

 It's our honor to talk about important issues and why they're relevant to each of us. The foundation of our show, Heart of the Matter, focuses on three things. Important issues and why they should be on our radar today. Looking at things as right versus wrong, instead of right versus left. And sharing stories with real people's experiences so that we can brainstorm solutions to problems that we're facing today. We know we can do this together. Thanks for listening and joining the conversation. Kim, how does somebody connect with us today? Well, on Facebook, we are the Americhicks. Our email is Americhicks at Gmail. Our website is americhicks.com. And you can listen live there and call us at 303-477-5600. Molly, we have a really great show plan for you today. Race relations and affirmative action will have Droy Murdoch with us at the half hour. And we saw him at Leadership Program in the Rockies. And it was just a really interesting presentation. And then we will have states in other L

### Stable whisper

La gran diferencia que tiene con whisper es que da escalas de tiempo mas exactas

In [2]:
import stable_whisper
model = stable_whisper.load_model('base')
result = model.transcribe("data/MSPCORPUS/Audio/MSP-Conversation_0002.wav")

C:\Users\emanu\Anaconda3\envs\SER\lib\site-packages\stable_whisper\whisper_word_level.py:224: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Transcribe:   0%|                                                                                                                                                                           | 0/1130.48 [00:00<?, ?sec/s]

Detected language: english


Transcribe: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1130.48/1130.48 [02:52<00:00,  6.54sec/s]


In [7]:
# Test de como guardar un json con la transcripcion
result.save_as_json('data/TRANSCRIPCIONES/audio.json')

Saved: C:\Users\emanu\Documents\GitHub\mspconv_ftlab\data\TRANSCRIPCIONES\audio.json


In [31]:
# Guardar el audio en distintos formatos
result.to_srt_vtt('audio.srt') #SRT
result.to_srt_vtt('audio.vtt') #VTT
result.to_ass('audio.ass') #ASS
result.to_tsv('audio.tsv') #TSV

Saved: C:\Users\emanu\Documents\GitHub\mspconv_ftlab\audio.srt
Saved: C:\Users\emanu\Documents\GitHub\mspconv_ftlab\audio.vtt
Saved: C:\Users\emanu\Documents\GitHub\mspconv_ftlab\audio.ass
Saved: C:\Users\emanu\Documents\GitHub\mspconv_ftlab\audio.tsv


In [4]:
# Test de como leer un json con la transcripción
result2 = stable_whisper.WhisperResult('audio.json')

In [6]:
result2[0][0]

WordTiming(word=" It's", start=0.0, end=0.16, probability=0.7852025926113129, tokens=[467, 311], left_locked=False, right_locked=False, segment_id=0, id=0)

## PELIGRO, NO CORRER, LOOP QUE GENERA LOS JSON DE TODAS LAS TRANSCRIPCIONES

1968

In [19]:
directory = 'data/MSPCORPUS/Audio'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f,filename)
        if int(filename.split('_')[1].split('.')[0]) >= 1968:
            result = model.transcribe(f)
            result.save_as_json(f'data/TRANSCRIPCIONES/{filename}.json')

data/MSPCORPUS/Audio\MSP-Conversation_0002.wav MSP-Conversation_0002.wav
data/MSPCORPUS/Audio\MSP-Conversation_0021.wav MSP-Conversation_0021.wav
data/MSPCORPUS/Audio\MSP-Conversation_0023.wav MSP-Conversation_0023.wav
data/MSPCORPUS/Audio\MSP-Conversation_0035.wav MSP-Conversation_0035.wav
data/MSPCORPUS/Audio\MSP-Conversation_0043.wav MSP-Conversation_0043.wav
data/MSPCORPUS/Audio\MSP-Conversation_0046.wav MSP-Conversation_0046.wav
data/MSPCORPUS/Audio\MSP-Conversation_0047.wav MSP-Conversation_0047.wav
data/MSPCORPUS/Audio\MSP-Conversation_0053.wav MSP-Conversation_0053.wav
data/MSPCORPUS/Audio\MSP-Conversation_0054.wav MSP-Conversation_0054.wav
data/MSPCORPUS/Audio\MSP-Conversation_0055.wav MSP-Conversation_0055.wav
data/MSPCORPUS/Audio\MSP-Conversation_0061.wav MSP-Conversation_0061.wav
data/MSPCORPUS/Audio\MSP-Conversation_0067.wav MSP-Conversation_0067.wav
data/MSPCORPUS/Audio\MSP-Conversation_0079.wav MSP-Conversation_0079.wav
data/MSPCORPUS/Audio\MSP-Conversation_0081.wav MSP-

Transcribe:   0%|                                                                                                                                                                            | 0/914.83 [00:00<?, ?sec/s]

Detected language: english


Transcribe: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 914.83/914.83 [01:57<00:00,  7.80sec/s]


Saved: C:\Users\emanu\Documents\GitHub\mspconv_ftlab\data\TRANSCRIPCIONES\MSP-Conversation_1968.wav.json
data/MSPCORPUS/Audio\MSP-Conversation_1992.wav MSP-Conversation_1992.wav


...

Saved: C:\Users\emanu\Documents\GitHub\mspconv_ftlab\data\TRANSCRIPCIONES\MSP-Conversation_2279.wav.json
data/MSPCORPUS/Audio\MSP-Conversation_2281.wav MSP-Conversation_2281.wav


Transcribe:   0%|                                                                                                                                                                           | 0/1188.68 [00:00<?, ?sec/s]

Detected language: english


Transcribe: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1188.68/1188.68 [02:21<00:00,  8.41sec/s]


Saved: C:\Users\emanu\Documents\GitHub\mspconv_ftlab\data\TRANSCRIPCIONES\MSP-Conversation_2281.wav.json
